In [1]:
from sklearn.datasets import fetch_california_housing

In [2]:
califonia = fetch_california_housing()

In [14]:
x = califonia.data
y = califonia.target

### 조건: Functional API
1. training set / validation set / test set 분리 (자율)
2. hidden layer 최소 3개 사용! (노드 수 자율)
3. early stopping 적용! (patience 7)
4. validation_data 옵션에 validation set 적용

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


In [15]:
x.shape, y.shape

((20640, 8), (20640,))

In [6]:
x_train = x[:int(x.shape[0]*0.6)]
x_val = x[x_train.shape[0]+1:int(x.shape[0]*0.8)]
x_test = x[x_train.shape[0] + x_val.shape[0]+1:int(x.shape[0])]

y_train = y[:int(y.shape[0]*0.6)]
y_val = y[y_train.shape[0]+1:int(y.shape[0]*0.8)]
y_test = y[y_train.shape[0] + y_val.shape[0]+1:int(y.shape[0])]

x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((12384, 8), (4127, 8), (4128, 8), (12384,), (4127,), (4128,))

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 2023)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2023)

x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((13209, 8), (3303, 8), (4128, 8), (13209,), (3303,), (4128,))

In [17]:
# 모델링: Functional API

# 1. 세션 클리어
keras.backend.clear_session()

# 2. 레이어 사슬처럼 엮기
il = keras.layers.Input(shape = (8, ))
hl1 = keras.layers.Dense(512, activation = 'relu')(il)
hl2 = keras.layers.Dense(256, activation = 'relu')(hl1)
hl3 = keras.layers.Dense(128, activation = 'relu')(hl2)
ol = keras.layers.Dense(1, activation = 'linear')(hl3)

# 3. 모델의 시작/끝 지정
model = keras.models.Model(il, ol)

# 4. 컴파일
model.compile(loss = 'mse', optimizer= 'adam')

# 요약
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 512)               4608      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 168,961
Trainable params: 168,961
Non-trainable params: 0
_________________________________________________________________


In [18]:
es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 7,
                   verbose = 1,
                   restore_best_weights= True)

In [19]:
model.fit(x_train, y_train,
          epochs = 50,
          verbose = 1,
          callbacks = [es],
          validation_data = (x_val, y_val),)

Epoch 1/50
413/413 [==============================] - 2s 3ms/step - loss: 201.7384 - val_loss: 8.6592
Epoch 2/50
413/413 [==============================] - 2s 4ms/step - loss: 25.2763 - val_loss: 1.2927
Epoch 3/50
413/413 [==============================] - 2s 4ms/step - loss: 4.7960 - val_loss: 1.4618
Epoch 4/50
413/413 [==============================] - 2s 4ms/step - loss: 13.3724 - val_loss: 2.3366
Epoch 5/50
413/413 [==============================] - 2s 4ms/step - loss: 5.1843 - val_loss: 123.1943
Epoch 6/50
413/413 [==============================] - 2s 4ms/step - loss: 5.7640 - val_loss: 0.7135
Epoch 7/50
413/413 [==============================] - 2s 4ms/step - loss: 1.1546 - val_loss: 0.7747
Epoch 8/50
413/413 [==============================] - 2s 4ms/step - loss: 3.8348 - val_loss: 0.7747
Epoch 9/50
413/413 [==============================] - 2s 4ms/step - loss: 2.9887 - val_loss: 1.2695
Epoch 10/50
413/413 [==============================] - 2s 4ms/step - loss: 2.1972 - val_loss: 

In [20]:
print(y_test)
y_pred = model.predict(x_test).reshape(-1,)
print(y_pred)

[3.656 2.01  2.132 ... 0.928 0.904 2.13 ]
129/129 [==============================] - 0s 2ms/step
[1.6830863  2.2340422  1.6341113  ... 1.8874787  0.95810497 2.3302965 ]


In [21]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred)

0.6439980184593768